In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# logger

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import logging

# 로거 생성 및 설정
logger = logging.getLogger()  # 기본 로거 가져오기
logger.setLevel(logging.INFO)  # 로그 수준 설정

# Jupyter Notebook에 출력될 수 있도록 스트림 핸들러 추가
if not logger.handlers:
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s %(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

# env

In [5]:
import boto3
import time

region_name = 'ap-northeast-2'

# ECS, CloudWatch 클라이언트 생성
ecs_client = boto3.client('ecs', region_name=region_name)
logs_client = boto3.client('logs')

2025-01-15 04:21:55,625 - INFO Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


# main

In [6]:
def run_task(conf):
    cluster = conf['cluster']
    task_definition = conf['task_definition']
    job_type = conf['job_type']
    
    project_hashkey = conf['project_hashkey']
    experiment_hashkey = conf['experiment_hashkey']
    
    table_name = conf['table_name']
    dataset_table_name = conf['dataset_table_name']
    dataset_profile_table_name = conf['dataset_profile_table_name']
    model_experiment_result_table_name = conf['model_experiment_result_table_name']
    model_repo_table_name = conf['model_repo_table_name']    
    
    username = conf['username']
    task_token = conf['task_token']
    account_id = conf['account_id']
    ecs_role_name = conf['ecs_role_name']
    subnet = conf['subnet']
    securityGroup = conf['securityGroup']
    cpu = str(conf['cpu'])
    memory = str(conf['memory'])
    
    task_role_arn = f'arn:aws:iam::{account_id}:role/{ecs_role_name}'
    container_name = task_definition

    # 작업 실행
    response = ecs_client.run_task(
        cluster = cluster,
        taskDefinition = task_definition,
        launchType='FARGATE',
        networkConfiguration={
            'awsvpcConfiguration': {
                'subnets': [
                    subnet,  # 실제 서브넷 ID로 대체
                ],
                'securityGroups': [
                    securityGroup,  # 실제 보안 그룹 ID로 대체
                ],
                'assignPublicIp': 'ENABLED'
            }
        },
        overrides={
            'taskRoleArn': task_role_arn,
            'containerOverrides': [
                {
                    'name': container_name,  # 실제 컨테이너 이름으로 대체
                    'command': [
                        'python', f'run_pm_{job_type}.py',
                        '--job_type', job_type,
                        '--dataset_table_name', dataset_table_name,
                        '--dataset_profile_table_name', dataset_profile_table_name,
                        '--table_name', table_name,
                        '--model_experiment_result_table_name', model_experiment_result_table_name,
                        '--model_repo_table_name', model_repo_table_name,
                        '--experiment_hashkey', experiment_hashkey,
                        '--project_hashkey', project_hashkey,
                        '--username', username,
                        '--task_token', task_token
                    ]
                }
            ],
            'cpu': cpu,  # CPU 유닛 (예: 512 = 0.5 vCPU)
            'memory': memory  # 메모리 (MB 단위, 예: 1024 = 1GB)
        }
    )
    return response

In [7]:
payload = {
    "cluster": "automl-fargate-cluster",
    "task_definition": "automl-fargate-papermill-tabular312-langchain",
    "table_name": "automl-regression-experiment",
    "project_hashkey": "da34ef08",
    "experiment_hashkey": "b13c616c",
    "dataset_table_name": "automl-dataset",
    "dataset_profile_table_name": "automl-dataset-profile-experiment-result",
    "model_repo_table_name": "automl-model-repo",
    "model_experiment_result_table_name": "automl-regression-experiment-result",
    "username": "sean@gs.co.kr",
    "job_type": "training",
    "cpu": 2048,
    "memory": 4096,
    "account_id": "405894843732",
    "ecs_role_name": "RoleStack-automlcdkprodEcsTaskExecutionRoleD78B1370-B8HLhToU7Ysq",
    "subnet": "subnet-09ba70ddf50da2846",
    "securityGroup": "sg-0e5468469f4239534"
}

In [10]:
payload["task_token"] = "123"

In [11]:
pp.pprint(payload)

{   'account_id': '405894843732',
    'cluster': 'automl-fargate-cluster',
    'cpu': 2048,
    'dataset_profile_table_name': 'automl-dataset-profile-experiment-result',
    'dataset_table_name': 'automl-dataset',
    'ecs_role_name': 'RoleStack-automlcdkprodEcsTaskExecutionRoleD78B1370-B8HLhToU7Ysq',
    'experiment_hashkey': 'b13c616c',
    'job_type': 'training',
    'memory': 4096,
    'model_experiment_result_table_name': 'automl-regression-experiment-result',
    'model_repo_table_name': 'automl-model-repo',
    'project_hashkey': 'da34ef08',
    'securityGroup': 'sg-0e5468469f4239534',
    'subnet': 'subnet-09ba70ddf50da2846',
    'table_name': 'automl-regression-experiment',
    'task_definition': 'automl-fargate-papermill-tabular312-langchain',
    'task_token': '123',
    'username': 'sean@gs.co.kr'}


In [12]:
response = run_task(payload)

# 실행 결과 출력
print(response)

{'tasks': [{'attachments': [{'id': '8493d0d0-7b20-4d88-b2fd-3bef612eb486', 'type': 'ElasticNetworkInterface', 'status': 'PRECREATED', 'details': [{'name': 'subnetId', 'value': 'subnet-09ba70ddf50da2846'}]}], 'attributes': [{'name': 'ecs.cpu-architecture', 'value': 'x86_64'}], 'availabilityZone': 'ap-northeast-2a', 'clusterArn': 'arn:aws:ecs:ap-northeast-2:405894843732:cluster/automl-fargate-cluster', 'containers': [{'containerArn': 'arn:aws:ecs:ap-northeast-2:405894843732:container/automl-fargate-cluster/ec44bed9d5cc443e818f8e05d2c81ac2/5c6f1e0a-87b6-43c6-a699-0680389f28d3', 'taskArn': 'arn:aws:ecs:ap-northeast-2:405894843732:task/automl-fargate-cluster/ec44bed9d5cc443e818f8e05d2c81ac2', 'name': 'automl-fargate-papermill-tabular312-langchain', 'image': '405894843732.dkr.ecr.ap-northeast-2.amazonaws.com/automl-fargate-papermill-tabular312-langchain:1.0', 'lastStatus': 'PENDING', 'networkInterfaces': [], 'cpu': '0'}], 'cpu': '2048', 'createdAt': datetime.datetime(2025, 1, 15, 4, 23, 22, 

# tail from cloudwatch

In [14]:
task_definition = payload['task_definition']

In [15]:
# 로그 그룹 이름 및 컨테이너 이름 설정
log_group_name = '/ecs/automl-fargate-papermill-tabular312-langchain'  # 실제 로그 그룹 이름으로 대체
container_name = task_definition

In [16]:
def tail_task_logs(log_group_name, log_stream_name):
    """
    CloudWatch 로그 그룹에서 특정 ECS Task의 로그를 tail 방식으로 출력하는 함수.
    
    Parameters:
    log_group_name (str): CloudWatch 로그 그룹 이름
    log_stream_name (str): CloudWatch 로그 스트림 이름
    """
    MAX_TIMEOUT = 60 * 10
    ts = time.time() + MAX_TIMEOUT
    next_token = None
    found_wrap_up = False
    retry_delay = 2  # 초기 재시도 대기 시간 (초)
    
    while not found_wrap_up:
        try:
            # 로그 가져오기
            if next_token:
                response = logs_client.get_log_events(
                    logGroupName=log_group_name,
                    logStreamName=log_stream_name,
                    nextToken=next_token,
                    startFromHead=False
                )
            else:
                response = logs_client.get_log_events(
                    logGroupName=log_group_name,
                    logStreamName=log_stream_name,
                    startFromHead=True
                )

            # 로그 이벤트 출력
            for event in response['events']:
                logger.info(event['message'])
                if 'wrap-up' in event['message']:
                    found_wrap_up = True
                    logger.info('Found "wrap-up".')
                    break
            
            if time.time() > ts:
                logger.info('Timeout reached without finding "wrap-up".')
                break
            
            # 다음 토큰 설정
            next_token = response['nextForwardToken']
            
            # 다음 로그 가져오기 전 대기
            time.sleep(2)
            retry_delay = 2  # 재시도 대기 시간 초기화

        except logs_client.exceptions.ResourceNotFoundException:
            if time.time() > ts:
                logger.error("Timeout reached while waiting for the log stream.")
                break
            logger.info("Log stream not found, waiting for provisioning...")
            time.sleep(retry_delay)
            retry_delay = min(retry_delay * 2, 30)  # 지수 백오프, 최대 30초
        except Exception as e:
            logger.error(f"Error while retrieving logs: {str(e)}")
            break

    return found_wrap_up

In [17]:
task_arn = response['tasks'][0]['taskArn']
task_id = task_arn.split('/')[-1]
log_stream_name = f'ecs/{task_definition}/{task_id}'

logger.info(task_id)
logger.info(log_stream_name)

2025-01-15 04:23:49,799 - INFO ec44bed9d5cc443e818f8e05d2c81ac2
2025-01-15 04:23:49,801 - INFO ecs/automl-fargate-papermill-tabular312-langchain/ec44bed9d5cc443e818f8e05d2c81ac2


In [18]:
tailed = tail_task_logs(log_group_name, log_stream_name)

2025-01-15 04:24:33,776 - INFO AWS Account ID: 405894843732
2025-01-15 04:24:33,776 - INFO AWS Region: ap-northeast-2
2025-01-15 04:24:33,777 - INFO analysis_llm.pkl 파일이 './work/training/input/analysis_llm.pkl'에 다운로드되었습니다.
2025-01-15 04:24:33,778 - INFO profile_df.pickle 파일이 './work/training/input/profile_df.pickle'에 다운로드되었습니다.
2025-01-15 04:24:33,778 - INFO upload/sean@gs.co.kr/a43a5c39/zip_file/common/llm.py 파일이 './work/common/llm.py'에 다운로드되었습니다.
2025-01-15 04:24:33,779 - INFO upload/sean@gs.co.kr/a43a5c39/zip_file/common/metric.py 파일이 './work/common/metric.py'에 다운로드되었습니다.
2025-01-15 04:24:33,780 - INFO upload/sean@gs.co.kr/a43a5c39/zip_file/common/modeling.py 파일이 './work/common/modeling.py'에 다운로드되었습니다.
2025-01-15 04:24:33,780 - INFO upload/sean@gs.co.kr/a43a5c39/zip_file/common/prep.py 파일이 './work/common/prep.py'에 다운로드되었습니다.
2025-01-15 04:24:33,781 - INFO upload/sean@gs.co.kr/a43a5c39/zip_file/common/utils.py 파일이 './work/common/utils.py'에 다운로드되었습니다.
2025-01-15 04:24:33,781 - INFO up